In [1]:
import os
import numpy as np
from zipfile import ZipFile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import confusion_matrix, classification_report

In [4]:
file_name = "Skin_Conditions_Aug.zip"

In [5]:
with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

Done


In [6]:
class_names = ['Acne', 'Carcinoma', 'Eczema', 'Keratosis', 'Milia', 'Rosacea']

In [7]:
base_dir = '/content/Skin_Conditions'
batch_size = 32
img_size = (224, 224)

In [8]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest',
    validation_split=0.2
)

In [9]:
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

Found 1920 images belonging to 6 classes.


In [10]:
validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


Found 474 images belonging to 6 classes.


In [11]:
base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

219055592/219055592 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [12]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

In [13]:
model = Model(inputs=base_model.input, outputs=predictions)

In [14]:
for layer in base_model.layers[-100:]:
    layer.trainable = True

In [15]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
checkpoint = ModelCheckpoint('inception_resnet_v2.keras', monitor='val_loss', save_best_only=True, mode='min')
early_stop = EarlyStopping(monitor='val_loss', patience=5, mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)

In [17]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=20,
    callbacks=[checkpoint, early_stop, reduce_lr]
)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


60/60 ━━━━━━━━━━━━━━━━━━━━ 2367s 37s/step - accuracy: 0.4479 - loss: 1.4155 - val_accuracy: 0.8103 - val_loss: 0.5644 - learning_rate: 1.0000e-04
Epoch 2/20


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


60/60 ━━━━━━━━━━━━━━━━━━━━ 10s 165ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.8462 - val_loss: 0.5894 - learning_rate: 1.0000e-04
Epoch 3/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 2296s 37s/step - accuracy: 0.8658 - loss: 0.4126 - val_accuracy: 0.8482 - val_loss: 0.5137 - learning_rate: 1.0000e-04
Epoch 4/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 44s 739ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.8077 - val_loss: 0.5395 - learning_rate: 1.0000e-04
Epoch 5/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 2295s 38s/step - accuracy: 0.9175 - loss: 0.2328 - val_accuracy: 0.8750 - val_loss: 0.3534 - learning_rate: 1.0000e-04
Epoch 6/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 82s 1s/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.9615 - val_loss: 0.0622 - learning_rate: 1.0000e-04
Epoch 7/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 2210s 37s/step - accuracy: 0.9342 - loss: 0.1790 - val_accuracy: 0.8839 - val_loss: 0.3249 - learning_rate: 1.0000e-04
Epoch 8/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 6s 105ms/s

In [18]:
loss, accuracy = model.evaluate(validation_generator)
print(f'\nValidation Loss: {loss}')
print(f'Validation Accuracy: {accuracy}')

15/15 ━━━━━━━━━━━━━━━━━━━━ 134s 9s/step - accuracy: 0.9563 - loss: 0.1580

Validation Loss: 0.20162148773670197
Validation Accuracy: 0.9388185739517212


In [19]:
y_true = validation_generator.classes
y_pred = model.predict(validation_generator)
y_pred_classes = np.argmax(y_pred, axis=1)

15/15 ━━━━━━━━━━━━━━━━━━━━ 150s 10s/step


In [20]:
cm = confusion_matrix(y_true, y_pred_classes)
cm

array([[15, 12, 12,  7, 14, 19],
       [14, 11, 11, 13, 20, 10],
       [16, 15, 11,  8, 15, 14],
       [13,  8, 14, 18, 14, 12],
       [12, 14, 12, 17, 15,  9],
       [16, 20, 11, 15,  6, 11]])

In [21]:
report = classification_report(y_true, y_pred_classes, target_names=class_names)
report

'              precision    recall  f1-score   support\n\n        Acne       0.17      0.19      0.18        79\n   Carcinoma       0.14      0.14      0.14        79\n      Eczema       0.15      0.14      0.15        79\n   Keratosis       0.23      0.23      0.23        79\n       Milia       0.18      0.19      0.18        79\n     Rosacea       0.15      0.14      0.14        79\n\n    accuracy                           0.17       474\n   macro avg       0.17      0.17      0.17       474\nweighted avg       0.17      0.17      0.17       474\n'